In [1]:
# import pandas as pd

In [2]:
# import snowflake.connector as sf

In [3]:
# from snowflake.connector import DictCursor

# from nltk.tokenize import RegexpTokenizer
# from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer

# from pprint import pprint
# import re

# # gensim (LDA)
# from gensim import corpora, models
# 
# from gensim.models import ldamodel as lda

# #spacy for lemmatization
# import spacy

# Imports

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
import pandas as pd
import numpy as np

import tqdm
tqdm.tqdm.pandas()

# Pull posts data from snowflake

In [7]:
from utils.get_data import get_posts

In [8]:
query = """
    SELECT distinct post_content || 'post_id: ' || post_id as post_content
        , post_id || '_' || site_prefix post_id
    FROM prod."posts"
    WHERE post_status = 'publish' and post_type = 'post'
    and site_prefix = 'WP'
    ORDER BY POST_ID DESC
    LIMIT 20000
"""

In [9]:
posts_df = get_posts(query)

In [10]:
posts_df

,POST_CONTENT,POST_ID
0,"As a big sister to four very sweet, hilarious,...",99918_WP
1,I spent my childhood growing up on a farm/ranc...,99917_WP
2,When am I going to get better at managing the ...,99841_WP
3,I know I am not alone in struggling to find a ...,99838_WP
4,"If you:\r\n<ul>\r\n \t<li>are new to migraine,...",99836_WP
...,...,...
3467,We asked our contributors a series of question...,100098_WP
3468,I feel as though I am always walking a fine li...,100073_WP
3469,"I have used all <a href=""/migraine-treatment/t...",100062_WP
3470,and flooded a shared back hallway between a re...,100037_WP


In [11]:

# #plotting tools
# import pyLDAvis
# import pyLDAvis.gensim
# import matplotlib.pyplot as plt
# %matplotlib inline

# #enable logging for gensim - optional
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning)

Cleaning Post Text
==================

In [12]:
from utils.clean_text import remove_html
from utils.clean_text import remove_between_square_brackets
from utils.clean_text import remove_post_id
from utils.clean_text import remove_backslash_symbols

In [13]:
posts_df.POST_CONTENT = posts_df.POST_CONTENT.progress_apply(lambda x: remove_html(x))
posts_df.POST_CONTENT = posts_df.POST_CONTENT.progress_apply(lambda x: remove_between_square_brackets(x))
posts_df.POST_CONTENT = posts_df.POST_CONTENT.progress_apply(lambda x: remove_post_id(x))
posts_df.POST_CONTENT = posts_df.POST_CONTENT.progress_apply(lambda x: remove_backslash_symbols(x))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3472/3472 [00:00<00:00, 93625.63it/s]


Removing Stopwords
================
Because we're going to be looking at descriptive words to determine the topics, we want to get rid of common english words that might bias the LDA's topic creations.  [This](https://gist.github.com/sebleier/554280) is a list of the stopwords currently included in the NLTK package. 

We are also removing 'migraine' from consideration as most posts will have it in the content somewhere, so including it in the LDA will cause most topics to contain the word, which weakens the coherence of the topics. 

# Post to word

In [14]:
dfList = posts_df['POST_CONTENT'].tolist()

In [15]:
from gensim.utils import simple_preprocess as gusp

def sent_to_words(sentences):
        for sentence in sentences:
            yield(gusp(str(sentence), deacc=True))
            #deacc removes punctuations
data_words = list(sent_to_words(posts_df['POST_CONTENT'].tolist()))

Bigrams & Trigrams
========
### Bi-grams and tri-grams allow us to combine two (bi) or three (tri) words that commonly appear together.



In [16]:
from gensim.models import Phrases as gmp
from gensim.models import phrases as gmpp

#build the models 
bigram=gmp(data_words, min_count=5, threshold=10)
#higher threshold = fewer phrases
trigram=gmp(bigram[data_words], threshold=10)

#faster way to get a sentence clubbed as trigram/bigram
bigram_mod=gmpp.Phraser(bigram)
trigram_mod=gmpp.Phraser(trigram)

In [17]:
from nltk.corpus import stopwords
stop_words=stopwords.words('english')
migraine_stopwords=['migraine', 'migraines', 'post_content', 'post_id']
stop_words.extend(migraine_stopwords)


#print(trigram_mod[bigram_mod[data_words[0]]])

##remove stopwords, make bigrams and lemmatize

#define functions for stopwords, bigrams, trigrams, & lemmatization
def remove_stopwords(texts):
    return[[word for word in gusp(str(doc)) if word not in stop_words] for doc in texts]
    
def make_bigrams(texts):
    return[bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return[trigram_mod[doc] for doc in texts]
    
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

##call functions in order

#remove stop words
data_words_nostops=remove_stopwords(data_words)

#form bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

#form trigrams
# data_words_trigrams = make_trigrams(data_words_bigrams)

In [18]:
import spacy
from gensim import corpora, models

#initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

#lemmatization - keep only noun, adj, verb, adv
data_lemmatized = lemmatization(data_words_bigrams)

##create dictionary and corpus needed for LDA
#dictionary
id2word = corpora.Dictionary(data_lemmatized)

#corpus
texts = data_lemmatized

#term doc frequency
corpus = [id2word.doc2bow(text) for text in texts]

#shows word frequency in document
# [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [19]:
from gensim.models import ldamodel as lda

#build topic model
#LDA model
lda_model = lda.LdaModel(corpus=corpus, id2word=id2word, num_topics=60, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)

doc_lda=lda_model[corpus]

In [20]:
lda_model.get_document_topics(corpus[0])

[(3, 0.016747354),
 (5, 0.25191197),
 (9, 0.05195418),
 (17, 0.045398846),
 (19, 0.011093096),
 (20, 0.02864925),
 (24, 0.010347577),
 (27, 0.15482982),
 (29, 0.016336074),
 (30, 0.30756164),
 (35, 0.01776216),
 (57, 0.02549048)]

In [21]:
posts_df['POST_CONTENT'].tolist()[0]

'As a big sister to four very sweet, hilarious, and very active younger brothers, I always want to provide fun and engaging entertainment whenever I spend time with them. I love to see a huge smile on their faces, and I love being able to provide or facilitate exciting environments and activities.That is why, despite my apprehension because of migraine...I volunteered to take my younger brother to his third basketball game of the season: loud cheering fans, bright lights, that super, incredibly obnoxious horn, screaming preteens, referee whistles and all. Sometimes it can be really tough to keep up with my kid brothers when I am experiencing pain from migraines, and even tougher to navigate that pain when I am in the role of caretaker for someone else.Living with chronic migraine can often entail being caught in situations in which many of us are having to choose between our family, friends, jobs, and livelihoods or migraine. Here, I am going to discuss how I navigate a trigger laden s

In [22]:
from pprint import pprint
pprint(lda_model.print_topics())

[(11,
  '0.000*"oversleeping" + 0.000*"pmi" + 0.000*"counteract" + 0.000*"lamplight" '
  '+ 0.000*"monologue" + 0.000*"brink" + 0.000*"accountsexpert" + 0.000*"turf" '
  '+ 0.000*"revery" + 0.000*"pmjim"'),
 (59,
  '0.000*"oversleeping" + 0.000*"pmi" + 0.000*"counteract" + 0.000*"lamplight" '
  '+ 0.000*"monologue" + 0.000*"brink" + 0.000*"accountsexpert" + 0.000*"turf" '
  '+ 0.000*"revery" + 0.000*"pmjim"'),
 (42,
  '0.000*"oversleeping" + 0.000*"pmi" + 0.000*"counteract" + 0.000*"lamplight" '
  '+ 0.000*"monologue" + 0.000*"brink" + 0.000*"accountsexpert" + 0.000*"turf" '
  '+ 0.000*"revery" + 0.000*"pmjim"'),
 (8,
  '0.096*"fat" + 0.076*"butter" + 0.000*"oversleeping" + 0.000*"turf" + '
  '0.000*"lamplight" + 0.000*"monologue" + 0.000*"continue" + 0.000*"pmi" + '
  '0.000*"accountsexpert" + 0.000*"pmjim"'),
 (18,
  '0.064*"hardly" + 0.041*"real_life" + 0.006*"consent" + 0.000*"brink" + '
  '0.000*"continue" + 0.000*"counteract" + 0.000*"bluntly" + 0.000*"lamplight" '
  '+ 0.000*"tu

In [23]:
from gensim.models import CoherenceModel

##compute perplexity and coherence
#perplexity - the lower the better -7.352 (-7.56 after removing HTML and [] content)
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

#coherence score 0.421 (.379 after removing HTML and [] content)
# 26 topics = 0.477
# 20 topics = 0.450
coherence_model_lda=CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda=coherence_model_lda.get_coherence()
print('\nCoherence Score:', coherence_lda)



Perplexity:  -19.349306367196295

Coherence Score: 0.4276171096514182


In [24]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [25]:
# #visualize LDA
# vis=pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
# vis

In [26]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=dfList):
    #init output
    sent_topics_df = pd.DataFrame()
    
    #get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list  
        row = sorted(row, key=lambda x:(x[1]), reverse=True)
        #get dominant topic, perc contribution, and keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j==0: # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_contribution', 'Topic_keywords']
    
    #add original text to the end
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

dfList = posts_df['POST_CONTENT'].tolist()
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=dfList)

#format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_no', 'Dominant_topic', 'Topic_perc_contrib', 'Keywords', 'Text']

#show
#df_dominant_topic.head(10)

In [27]:
df_dominant_topic.Keywords.nunique()

23

In [28]:
df_dominant_topic.Keywords.value_counts()

way, pain, many, life, well, thing, often, even, also, other                                                         1576
day, time, pain, still, really, long, year, first, back, bad                                                         1319
patient, treatment, pain, cluster_headache, also, risk, cluster, result, device, dose                                 233
medication, drug, treatment, side_effect, medicine, new, prescription, relief, injection, effective                    76
new, information, health, research, state, condition, many, insurance, issue, specialist                               64
community, com, share, event, online, video, member, article, advocate, awareness                                      45
attack, symptom, nausea, experience, also, severe, often, aura, oxygen, episode                                        37
headache, diagnosis, individual, condition, disorder, type, age, problem, pain, common                                 27
study, cgrp, clinical_tr

In [29]:
df_dominant_topic

,Document_no,Dominant_topic,Topic_perc_contrib,Keywords,Text
0,0,30.0,0.3076,"water, fun, hot, caretaker, cold, show, music,...","As a big sister to four very sweet, hilarious,..."
1,1,5.0,0.2463,"day, time, pain, still, really, long, year, fi...",I spent my childhood growing up on a farm/ranc...
2,2,27.0,0.4332,"way, pain, many, life, well, thing, often, eve...",When am I going to get better at managing the ...
3,3,27.0,0.2939,"way, pain, many, life, well, thing, often, eve...",I know I am not alone in struggling to find a ...
4,4,27.0,0.4152,"way, pain, many, life, well, thing, often, eve...","If you:are new to migraine, having been recent..."
...,...,...,...,...,...
3467,3467,5.0,0.2607,"day, time, pain, still, really, long, year, fi...",We asked our contributors a series of question...
3468,3468,27.0,0.3835,"way, pain, many, life, well, thing, often, eve...",I feel as though I am always walking a fine li...
3469,3469,27.0,0.1756,"way, pain, many, life, well, thing, often, eve...",I have used all triptans extensively over 25 y...
3470,3470,5.0,0.4097,"day, time, pain, still, really, long, year, fi...",and flooded a shared back hallway between a re...


In [32]:
#most representative doc for each topic
#group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, grp.sort_values(['Perc_contribution'], ascending=[0]).head(1)], axis=0)

#reset index
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

#format
sent_topics_sorteddf_mallet.columns = ['Topic_num', 'Topic_perc_contrib', 'keywords', 'text']

sent_topics_sorteddf_mallet

# sent_topics_sorteddf_mallet.to_csv('LDA/most_rep_post_lda.csv')

,Topic_num,Topic_perc_contrib,keywords,text
0,0.0,0.4351,"food, trigger, diet, product, supplement, vita...",Fermented foods (and beverages) have been a pa...
1,1.0,0.1866,"mile, race, pee, proper_diagnosis, philadelphi...",January 2018 Update: Check out the 2018 Miles ...
2,2.0,0.5677,"community, com, story, share, article, informa...","Throughout the year, hundreds of our community..."
3,3.0,0.5540,"attack, symptom, pain, headache, often, also, ...","For those suffering from migraines, the number..."
4,4.0,0.2486,"device, hair, dog, refill, band, technology, b...",I want to talk about one of my new favorite bo...
5,5.0,0.7841,"day, time, pain, really, still, long, back, he...",A couple of days ago I mentioned weird shootin...
6,8.0,0.8266,"promotion, health_union, sponsor, right, winne...","Official Rules, Terms and Conditions for the H..."
7,10.0,0.2421,"child, kid, parent, school, habit, adult, holi...","""I feel so guilty for what my migraine attacks..."
8,13.0,0.6224,"time, good, work, well, thing, also, help, hea...",I want to share some advice I've learned about...
9,14.0,0.5598,"body, brain, wife, part, state, son, baby, act...","Surf’s upRiding a wave toward the shore, the p..."


/Users/abratun/Repos/KeywordExtraction/venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/abratun/Repos/KeywordExtraction/venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/abratun/Repos/KeywordExtraction/venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/abratun/Repos/KeywordExtraction/venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/a